In [1]:
# 라이브러리 로딩

from pandas_ods_reader import read_ods
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as W
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import argparse
import getpass
import json
import pandas as pd
import pprint
import sys
import time

# 원 요청파일에서 IP 가져오기

In [2]:
base_path = r"./21년도 정보시스템 및 인프라 정기점검_11월02일기준_모니터링(전체).ods"
sheet = 'Sheet'

df2 = read_ods(base_path, sheet, headers=True)

df2 = df2[(df2["담당팀명"] == '클라우드운영센터') & (df2["점검상태.1"] == "미점검")]

df2 = df2[['HOSTNAME', 'IP', '담당팀명', '담당자명', '점검상태.1']]

df2 = df2.set_axis(["hostname", 'ip', "team", "owner", "status"], axis=1)

df2.reset_index(inplace=True)

In [3]:
df2

,index,hostname,ip,team,owner,status
0,1071,ccskrccspdsrediss01.krcloud_redisDB,10.11.52.235,클라우드운영센터,김의현,미점검
1,1072,ccskrccspdsrediss03.krcloud_redisDB,10.11.52.237,클라우드운영센터,김의현,미점검
2,1077,ccskrccspdsredisp02.krtms_redisDB,10.11.57.55,클라우드운영센터,김의현,미점검
3,1078,ccskrccspdsredisp03.krtms_redisDB,10.11.57.56,클라우드운영센터,김의현,미점검
4,1082,ccskrccspdsredisp01.krtms_redisDB,10.11.57.54,클라우드운영센터,김의현,미점검
...,...,...,...,...,...,...
378,8571,krcloudstgcomdr311.krcloud,10.11.49.141,클라우드운영센터,장재봉,미점검
379,8575,krcloudstgcomdr312.krcloud,10.11.49.142,클라우드운영센터,장재봉,미점검
380,8577,krcloudstgcomdr310.krcloud,10.11.49.140,클라우드운영센터,장재봉,미점검
381,8578,krcloudstgcomdr307.krcloud,10.11.49.137,클라우드운영센터,장재봉,미점검


# 진단실행 보고서 읽어오기

In [4]:
base_path = r"./tasks_04.ods"
sheet = 'tasks_04'

In [5]:
df = read_ods(base_path, sheet, headers=True)

In [6]:
df

,unnamed.1,RDS-501,RDS-201,RDS-301,RDS-101,RDS-102,RDS-202,RDS-402,RDS-401,RDS-403,...,U-102,U-109,U-205,U-304,U-309,U-310,U-404,U-407,U-412,U-413
0,ccskrccspdsrediss01,failed,success,success,success,success,success,success,success,success,...,None,None,None,None,None,None,None,None,None,None
1,ccskrccspdsrediss03,failed,success,success,success,success,success,success,success,success,...,None,None,None,None,None,None,None,None,None,None
2,ccskrccspdsredisp02,success,success,success,success,success,failed,success,success,success,...,None,None,None,None,None,None,None,None,None,None
3,ccskrccspdsredisp03,success,success,success,success,success,failed,success,success,success,...,None,None,None,None,None,None,None,None,None,None
4,ccskrccspdsredisp01,success,success,success,success,success,failed,success,success,success,...,None,None,None,None,None,None,None,None,None,None
5,ccskrccspdsrediss02,failed,success,success,success,success,success,success,success,success,...,None,None,None,None,None,None,None,None,None,None
6,ocdb01.hkmc,None,None,None,None,None,None,None,None,None,...,success,success,success,success,success,success,success,success,success,success
7,ocapi01,None,None,None,None,None,None,None,None,None,...,success,success,success,success,success,success,success,success,success,success
8,occtrl01.hkmc,None,None,None,None,None,None,None,None,None,...,success,success,success,success,success,success,success,success,success,success
9,ocosd01.hkmc,None,None,None,None,None,None,None,None,None,...,success,success,success,success,success,success,success,success,success,success


# Inventory file 생성하기

In [7]:
df.rename(columns = { 'unnamed.1' : 'hostname' }, inplace=True)

In [8]:
df

,hostname,RDS-501,RDS-201,RDS-301,RDS-101,RDS-102,RDS-202,RDS-402,RDS-401,RDS-403,...,U-102,U-109,U-205,U-304,U-309,U-310,U-404,U-407,U-412,U-413
0,ccskrccspdsrediss01,failed,success,success,success,success,success,success,success,success,...,None,None,None,None,None,None,None,None,None,None
1,ccskrccspdsrediss03,failed,success,success,success,success,success,success,success,success,...,None,None,None,None,None,None,None,None,None,None
2,ccskrccspdsredisp02,success,success,success,success,success,failed,success,success,success,...,None,None,None,None,None,None,None,None,None,None
3,ccskrccspdsredisp03,success,success,success,success,success,failed,success,success,success,...,None,None,None,None,None,None,None,None,None,None
4,ccskrccspdsredisp01,success,success,success,success,success,failed,success,success,success,...,None,None,None,None,None,None,None,None,None,None
5,ccskrccspdsrediss02,failed,success,success,success,success,success,success,success,success,...,None,None,None,None,None,None,None,None,None,None
6,ocdb01.hkmc,None,None,None,None,None,None,None,None,None,...,success,success,success,success,success,success,success,success,success,success
7,ocapi01,None,None,None,None,None,None,None,None,None,...,success,success,success,success,success,success,success,success,success,success
8,occtrl01.hkmc,None,None,None,None,None,None,None,None,None,...,success,success,success,success,success,success,success,success,success,success
9,ocosd01.hkmc,None,None,None,None,None,None,None,None,None,...,success,success,success,success,success,success,success,success,success,success


In [10]:
hostnames = df[df.columns[0]]
hostnames

0     ccskrccspdsrediss01
1     ccskrccspdsrediss03
2     ccskrccspdsredisp02
3     ccskrccspdsredisp03
4     ccskrccspdsredisp01
5     ccskrccspdsrediss02
6             ocdb01.hkmc
7                 ocapi01
8           occtrl01.hkmc
9            ocosd01.hkmc
10           ocosd02.hkmc
Name: hostname, dtype: object

In [11]:
for h in hostnames:
    for i, row in df2.iterrows():
        if h in row['hostname']:
            print(h, '\t', row['ip'])

ccskrccspdsrediss01 	 10.11.52.235
ccskrccspdsrediss03 	 10.11.52.237
ccskrccspdsredisp02 	 10.11.57.55
ccskrccspdsredisp03 	 10.11.57.56
ccskrccspdsredisp01 	 10.11.57.54
ccskrccspdsrediss02 	 10.11.52.236
ocdb01.hkmc 	 10.12.17.32
ocapi01 	 10.12.0.33
occtrl01.hkmc 	 10.12.17.34
ocosd01.hkmc 	 10.12.18.1
ocosd02.hkmc 	 10.12.18.2


### 화면에 출력해 보기

In [12]:
hostnames = df[df.columns[0]]

for t in range(1, len(df.columns)):
    result = df[df.columns[t]]
    print("[" + df.columns[t] + "]")
    for h, e in zip(hostnames, result):      
        if e == 'failed':
            for i, row in df2.iterrows():
                if h in row['hostname']:
                    print(h, '\t', 'ansible_host='+row['ip'])
    print("\n")

[RDS-501]
ccskrccspdsrediss01 	 ansible_host=10.11.52.235
ccskrccspdsrediss03 	 ansible_host=10.11.52.237
ccskrccspdsrediss02 	 ansible_host=10.11.52.236


[RDS-201]


[RDS-301]


[RDS-101]


[RDS-102]


[RDS-202]
ccskrccspdsredisp02 	 ansible_host=10.11.57.55
ccskrccspdsredisp03 	 ansible_host=10.11.57.56
ccskrccspdsredisp01 	 ansible_host=10.11.57.54


[RDS-402]


[RDS-401]


[RDS-403]


[RDS-203]


[RDS-302]


[U-303]
ocdb01.hkmc 	 ansible_host=10.12.17.32
occtrl01.hkmc 	 ansible_host=10.12.17.34
ocosd01.hkmc 	 ansible_host=10.12.18.1
ocosd02.hkmc 	 ansible_host=10.12.18.2


[U-103]


[U-104]


[U-106]


[U-1403]


[U-301]


[U-302]
ocapi01 	 ansible_host=10.12.0.33


[U-401]


[U-403]


[U-406]


[U-602]


[U-101]


[U-105]


[U-107]


[U-108]


[U-1402]


[U-201]


[U-202]


[U-203]


[U-204]


[U-305]


[U-306]


[U-307]


[U-308]


[U-405]


[U-408]


[U-409]


[U-410]


[U-411]


[U-501]


[U-601]


[U-102]


[U-109]


[U-205]


[U-304]


[U-309]


[U-310]


[U-404]


[U-407]



### 파일로 내보내기

In [13]:
hostnames = df[df.columns[0]]

with open("i-sams.tasks.inventory", "w") as text_file:
    for t in range(1, len(df.columns)):
        result = df[df.columns[t]]
        print("[" + df.columns[t] + "]", file=text_file)
        for h, e in zip(hostnames, result):      
            if e == 'failed':
                for i, row in df2.iterrows():
                    if h in row['hostname']:
                        print(h, '\t', 'ansible_host='+row['ip'], file=text_file)
        print("\n", file=text_file)